In [233]:
import os
import re

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
HOME = ".."
DATA_DIR = "data"
NYSE_FUNDAMENTALS = os.path.join(HOME, DATA_DIR, "nyse", "fundamentals.csv")
NYSE_PRICES = os.path.join(HOME, DATA_DIR, "nyse", "prices.csv")
NYSE_PRICES_SPLIT = os.path.join(HOME, DATA_DIR, "nyse", "prices-split-adjusted.csv")
NYSE_SECURITIES = os.path.join(HOME, DATA_DIR, "nyse", "securities.csv")

In [204]:
fund = pd.read_csv(NYSE_FUNDAMENTALS)
securities = pd.read_csv(NYSE_SECURITIES)
prices = pd.read_csv(NYSE_PRICES)
prices_split = pd.read_csv(NYSE_PRICES_SPLIT)

In [219]:
prices['date'] = pd.to_datetime(prices['date'], errors='coerce')  # [datetime.strptime(x[:10], '%Y-%m-%d') for x in prices['date']]
prices_split['date'] = pd.to_datetime(prices_split['date'], errors='coerce')
securities['Date first added'] = pd.to_datetime(securities['Date first added'], errors='coerce')

### Duplicate securities for one company

Stock classes are used to distribute shares with different voting rights. e.g. Googles founders (Class B) have more voting rights as with normal shares (Class A). Class C is defined for shares without voting rights.

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Recent_changes_to_the_list_of_S&P_500_Components

##### Other observations:
- Table just copied from Wikipedia (presumably at the end of 2016)
- Some security names are missing Inc./Corp. missing (verified via Wikipedia)
- Date first added is sometimes undefined (also in Wikipedia table)
- The state might have changed between 2010 & 2016. Take this into consideration?

In [227]:
x = securities.copy()
idx = securities.index[securities['Ticker symbol'] == 'DISCA'][0]
x.loc[idx, 'Security'] = 'Discovery Communications Class A'  # Before: Discovery Communications-A
idx = securities.index[securities['Ticker symbol'] == 'DISCK'][0]
x.loc[idx, 'Security'] = 'Discovery Communications Class C'  # Before: Discovery Communications-C
idx = securities.index[securities['Ticker symbol'] == 'UA'][0]
x.loc[idx, 'Security'] = 'Under Armour Class C'  # Before: Under Armour
idx = securities.index[securities['Ticker symbol'] == 'UAA'][0]
x.loc[idx, 'Security'] = 'Under Armour Class A'  # Before: Under Armour

In [228]:
columns = x.columns.tolist()
columns.insert(1, 'Name')

regex_pat = re.compile(r'\WClass (A|B|C)$', flags=re.IGNORECASE)
x['Name'] = x['Security'].str.replace(regex_pat, '')

x = x[columns]  # Change order

In [39]:
securities[securities['Ticker symbol'] == 'AMZN']

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
26,AMZN,Amazon.com Inc,reports,Consumer Discretionary,Internet & Direct Marketing Retail,"Seattle, Washington",2005-11-18,1018724


In [244]:
prices[prices['symbol'] == 'AMZN'].head()

,date,symbol,open,close,low,high,volume
284,2010-01-04,AMZN,136.250000,133.899994,133.139999,136.610001,7599900.0
751,2010-01-05,AMZN,133.429993,134.690002,131.809998,135.479996,8851900.0
1219,2010-01-06,AMZN,134.600006,132.250000,131.649994,134.729996,7178800.0
1687,2010-01-07,AMZN,132.009995,130.000000,128.800003,132.320007,11030200.0
2155,2010-01-08,AMZN,130.559998,133.520004,129.029999,133.679993,9830500.0


In [238]:
import nyse

In [240]:
nyse.load()

In [241]:
nyse.get_securities_problems()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
23,GOOGL,Alphabet Inc Class A,reports,Information Technology,Internet Software & Services,"Mountain View, California",2014-04-03,1652044
24,GOOG,Alphabet Inc Class C,reports,Information Technology,Internet Software & Services,"Mountain View, California",NaN,1652044
142,DISCA,Discovery Communications-A,reports,Consumer Discretionary,Cable & Satellite,"Silver Spring, Maryland",2010-03-01,1437107
143,DISCK,Discovery Communications-C,reports,Consumer Discretionary,Cable & Satellite,"Silver Spring, Maryland",2014-08-07,1437107
326,NWSA,News Corp. Class A,reports,Consumer Discretionary,Publishing,"New York, New York",2013-08-01,1564708
327,NWS,News Corp. Class B,reports,Consumer Discretionary,Publishing,"New York, New York",2015-09-18,1564708
449,FOXA,Twenty-First Century Fox Class A,reports,Consumer Discretionary,Publishing,"New York, New York",2013-07-01,1308161
450,FOX,Twenty-First Century Fox Class B,reports,Consumer Discretionary,Publishing,"New York, New York",2015-09-18,1308161
455,UA,Under Armour,reports,Consumer Discretionary,"Apparel, Accessories & Luxury Goods","Baltimore, Maryland",2014-05-01,1336917
456,UAA,Under Armour,reports,Consumer Discretionary,"Apparel, Accessories & Luxury Goods","Baltimore, Maryland",2016-03-25,1336917


In [236]:
nyse.get_fundamentals('UA')

({'UA': {'exchange': 'NYQ',
   'shortName': 'Under Armour, Inc. Class C',
   'longName': 'Under Armour, Inc.',
   'exchangeTimezoneName': 'America/New_York',
   'exchangeTimezoneShortName': 'EST',
   'isEsgPopulated': False,
   'gmtOffSetMilliseconds': '-18000000',
   'underlyingSymbol': None,
   'quoteType': 'EQUITY',
   'symbol': 'UA',
   'underlyingExchangeSymbol': None,
   'headSymbol': None,
   'messageBoardId': 'finmb_8740684',
   'uuid': '3943d879-5e03-38e3-b07b-5b71eac1b669',
   'market': 'us_market'}},
 {'balanceSheetHistory': {'UA': [{'2017-12-31': {'intangibleAssets': 46995000,
      'capitalSurplus': 872266000,
      'totalLiab': 1987725000,
      'totalStockholderEquity': 2018642000,
      'otherCurrentLiab': 50426000,
      'totalAssets': 4006367000,
      'commonStock': 146000,
      'retainedEarnings': 1184441000,
      'otherLiab': 162304000,
      'goodWill': 555674000,
      'treasuryStock': -38211000,
      'otherAssets': 166445000,
      'cash': 312483000,
      't